In [53]:
import ipyvuetify as v
import ipywidgets as widgets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from IPython.display import display, clear_output
import json

### SQLytes Inventory Tracking



In [54]:
backend = 'http://127.0.0.1:5000/sqlytes/'
# output_area = widgets.Output()

### Select Warehouse to search the parts for

In [56]:
warehouseWidgets = WarehouseWidgets()

In [57]:
# warehouseWidgets.warehouses_options_dropdown.observe(warehouseWidgets.warehouse_options_handler,names='value')

In [58]:
# Display the dropdown initially
#display(warehouseWidgets.warehouses_options_dropdown)

In [59]:
warehouseWidgets.getWarehousePartsAPI(warehouseWidgets.warehouse_selection.value)
warehouseWidgets.warehouse_selection.observe(warehouseWidgets.on_selection_change)
display(warehouseWidgets.warehouse_selection)
display(warehouseWidgets.output_area)

Dropdown(description='Select wid:', options=(3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 1, 9, 2), value=3)

Output()

In [60]:
# display(warehouseWidgets.warehouses_options_dropdown)

In [61]:
# warehouseWidgets.displayNextWarehouse(_)

In [62]:
# # Attach the function to the button's click event
# button = warehouseWidgets.next_warehouse_button
# button.on_click(warehouseWidgets.displayNextWarehouse)

# # Display the widgets
# display(warehouseWidgets.output_area)
# display(button)

### Select supplier to search the parts it supplies

In [63]:
suppliedPartsWidgets = SuppliedPartsWidgets()

In [64]:
suppliedPartsWidgets.getSuppliedPartsAPI(suppliedPartsWidgets.supplier_selection.value)
suppliedPartsWidgets.supplier_selection.observe(suppliedPartsWidgets.on_selection_change)
display(suppliedPartsWidgets.supplier_selection)
display(suppliedPartsWidgets.output_area)

Dropdown(description='Select sid:', options=(1, 2, 3, 4, 5, 6, 7), value=1)

Output()

# Transactions

In [93]:
class TransactionWidgets:
    def __init__(self):
        self.output_area = widgets.Output()
        self.available_warehouses = self.getAvailableWarehouses()
        self.warehouses = self.getAllWarehouses()
        self.current_warehouse_index = 0
        self.all_transactions_df = pd.DataFrame()
       
        self.warehouse_options_dropdown = widgets.Select(
            options=['All Warehouses info', 'All transactions in this warehouse'],
            value='All Warehouses info',
            description='Search for:',
            rows=2,
        )

        self.warehouse_selection = widgets.Dropdown(
            options=self.available_warehouses,
            value=self.available_warehouses[0],
            description="Select sid:"
        )
        
        self.warehouse_selection.observe(self.on_selection_change)
        
        self.next_warehouse_button = widgets.Button(
            description="Next Warehouse",
            icon='check'
        )
        

    def getAvailableWarehouses(self):
        """
        Return all IDs of the warehouses in the DB
        """
        response = requests.get(f'{backend}{"warehouse"}')
        # List to store all available sid's.
        wids = []
        # Check if response was succesful.
        if response.status_code == 200:
            # Parse JSON Data from response.
            data = response.json()
            for record in data['Warehouses']:
                wids.append(record['wid'])
        else:
            print('Error {}: Could not get data from the backend.'.format(response.status_code))
        return sorted(wids)

    
    def getAllWarehouses(self):
        """
        Return all suppliers in the Data Base.
        """
        response = requests.get(f'{backend}{"warehouse"}')
        # Check if response was succesful.
        if response.status_code == 200:
            # Parse JSON Data from response.
            data = response.json()
            return data
        print(f'Error {response.status_code}: Could not get data from the backend.')
        return {}
                
    def getTransactionsPerWarehouseAPI(self, selected_warehouse):
        endpoint = f'transaction'
        response = requests.put(f'{backend}{endpoint}', data=json.dumps({"wid": selected_warehouse}), headers={"Content-Type": "application/json"})
        with self.output_area:
            clear_output(wait=True)  # Clear the previous output
            if response.status_code == 200: 
                data = response.json()
                supplied_parts_df = pd.DataFrame(data["Result"])
                display(supplied_parts_df)
            elif response.status_code == 404:
                print('Error {}: Warehouse does not have any transactions'.format(response.status_code))
            else:
                print(f'{backend}{endpoint}')
                print('Error {}: Could not get data from the backend.'.format(response.status_code))
                
    def on_selection_change(self,change):
        if isinstance(change, int):
            # Directly received the selected value, not an event dictionary
            selected_warehouse = change
        elif change['type'] == 'change' and change['name'] == 'value':
            # Received an event dictionary, extract the selected value
            selected_warehouse    = change['new']
        else:
            # Invalid change, do nothing
            return
        self.getTransactionsPerWarehouseAPI(selected_warehouse)


In [97]:
transactionWidget = TransactionWidgets()
transactionWidget.getTransactionsPerWarehouseAPI(transactionWidget.warehouse_selection.value)
transactionWidget.warehouse_selection.observe(transactionWidget.on_selection_change)
display(transactionWidget.warehouse_selection)
display(transactionWidget.output_area)

Dropdown(description='Select sid:', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14), value=1)

Output()